Import Libraries

Define ODEs system

In [ ]:
# Define the system of ODEs
def system_ode(y, t, b, D1, D2, D3, Beta, Alpha, Mu, Eta):
    S, I1, I2, R = y

    # Equations
    dS_dt = b - (Beta * S * I1) - (D1 * S)
    dI1_dt = (Beta * S * I1) - ((D1 + D2 + Alpha) * I1) + (Eta * I1)
    dI2_dt = (Alpha * I1) - ((D2 + D3 + D3 + Mu) * I2)
    dR_dt = (Mu * I2) - ((D1 + D2) * R)

    return [dS_dt, dI1_dt, dI2_dt, dR_dt]

Simulate System Dynamics

In [ ]:
# Generate synthetic data
def generate_data(params, initial_conditions, t):
    return odeint(system_ode, initial_conditions, t, args=params)

# Normalize data
def normalize(data, data_min=None, data_max=None):
    if data_min is None or data_max is None:
        data_min, data_max = np.min(data, axis=0), np.max(data, axis=0)
    return (data - data_min) / (data_max - data_min), data_min, data_max

# Denormalize data
def denormalize(data, data_min, data_max):
    return data * (data_max - data_min) + data_min

Train the PINN model

In [ ]:
 # Training function
def train_pinn(model, X_train, Y_train, epochs=5000, learning_rate=1e-5):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_history = []

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predictions = model(X_train)
            loss = tf.reduce_mean(tf.square(predictions - Y_train))

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        loss_history.append(loss.numpy())
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

    return loss_history

Define parameters

In [ ]:

# Generate data and normalize
synthetic_data = generate_data(params, initial_conditions, t)
synthetic_data, y_min, y_max = normalize(synthetic_data)
t, t_min, t_max = normalize(t.reshape(-1, 1))

# Convert to tensors
X_train = tf.convert_to_tensor(t, dtype=tf.float32)
Y_train = tf.convert_to_tensor(synthetic_data, dtype=tf.float32)


plot Loss

In [ ]:

# Initialize and train the PINN

# Plot training loss
plt.plot(loss_history)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

# Predict and denormalize results
predictions = model(X_train).numpy()
predictions = denormalize(predictions, y_min, y_max)
synthetic_data = denormalize(synthetic_data, y_min, y_max)

# Plot results
labels = ['S', 'I1', 'I2', 'R']
for i in range(4):
    plt.figure(figsize=(10, 6))
    plt.plot(denormalize(t, t_min, t_max), synthetic_data[:, i], label="Ground Truth")
    plt.plot(denormalize(t, t_min, t_max), predictions[:, i], "--", label="PINN Prediction")
    plt.xlabel("Time (days)")
    plt.ylabel(labels[i])
    plt.legend()
    plt.title(f"{labels[i]} Population Dynamics")
    plt.show()